# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Now that we've learned about multi-armed bandits and methods for optimizing rewards, let's look at real-world applications, starting with a stock market example. We'll also learn a little more about configuring RLlib trainers.

We'll load a dataset derived from this [NYU Stern table](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjustments for inflation. The `market.tsv` file in this folder contains the data.

In [1]:
import pandas as pd
import numpy as np
import os, sys

In [2]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + '/market.tsv'  # full path

In [3]:
def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [4]:
df = load_market_data(DEFAULT_DATA_FILE)
df

,year,inflation,sp500,t.bill,t.bond,corp
0,1928,-1.15,45.49,4.28,2.01,4.42
1,1929,0.00,-8.30,3.16,4.20,3.02
2,1930,-2.67,-23.07,7.42,7.41,3.30
3,1931,-8.93,-38.33,12.34,7.00,-7.41
4,1932,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...
87,2015,0.12,1.26,-0.07,1.16,-0.82
88,2016,1.26,10.38,-0.93,-0.56,8.99
89,2017,2.13,19.07,-1.17,0.66,7.44
90,2018,2.44,-6.51,-0.49,-2.40,-5.08


As you can see the data spans 92 years, from 1928 to 2019. The columns represent:
  * the year
  * inflation rate at the time
  * [S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index) (composite stock index)
  * [Treasury Bills](https://www.investopedia.com/terms/t/treasurybill.asp) (short-term gov bonds)
  * [Treasury Bonds](https://www.investopedia.com/terms/t/treasurybond.asp) (long-term gov bonds)
  * [Moody's Baa Corporate Bonds](https://en.wikipedia.org/wiki/Moody%27s_Investors_Service#Moody's_credit_ratings) (moderate risk)

In [5]:
df.describe()

,year,inflation,sp500,t.bill,t.bond,corp
count,92.00000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,1973.50000,3.041957,8.413261,0.434239,2.166413,4.216630
std,26.70206,3.803579,19.619605,3.573035,8.126432,8.625809
min,1928.00000,-10.300000,-38.900000,-12.050000,-14.570000,-14.850000
25%,1950.75000,1.415000,-2.740000,-1.185000,-2.620000,-1.322500
50%,1973.50000,2.750000,10.515000,0.590000,1.070000,3.910000
75%,1996.25000,4.275000,20.622500,2.117500,7.037500,9.287500
max,2019.00000,14.390000,58.200000,12.680000,25.140000,37.780000


"Corp" refers to corporate bonds.

## Analysis of the Data

What are the worst case and best case scenarios? In other words, if one could predict the future market performance, what are the possible ranges of total failure vs. total success over the past century? By "total", we mean what if you had all your money in a given year invested in the worst performing _sector_ (S&P500, T bills, or other) or you were invested in the best performing sector for that year.

In [6]:
n_years = len(df)
min_list = []
max_list = []

for i in range(n_years):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_years))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_years))

-5.64% worst case annualized
15.18% best case annualized


In [7]:
sys.path.append('../..')
from util.line_plots import plot_line, plot_line_with_stddev, plot_between_lines
from bokeh_util import plot_cumulative_regret

In [8]:
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [9]:
min_max = pd.DataFrame.from_dict({'year': df['year'], 'min':min_list, 'max':max_list})
min_max

,year,min,max
0,1928,2.01,45.49
1,1929,-8.30,4.20
2,1930,-23.07,7.42
3,1931,-38.33,12.34
4,1932,1.85,37.78
...,...,...,...
87,2015,-0.82,1.26
88,2016,-0.93,10.38
89,2017,-1.17,19.07
90,2018,-6.51,-0.49


In [10]:
plot_between_lines(min_max, x_col='year', lower_col='min', upper_col='max', 
                   title='Best to Worst', x_axis_label='year', y_axis_label='%')

From the graph, there are some years where the performance varies widely, while other years everything returns about the same performance.

## Defining an Environment

Now let's define a Gym environment so that we can train a contextual bandit to optimize annual investments over that period.

In [11]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import numpy as np
import random

This is the bandit we'll use to represent the market "environment".

In [12]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = config.get('max-inflation', DEFAULT_MAX_INFLATION)
        self.tickers = config.get('tickers', DEFAULT_TICKERS)
        self.data_file = config.get('data-file', DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

Let's see it in action:

In [13]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /home/paul/academy/ray-rllib/multi-armed-bandits/market.tsv (config: {})
2 [[-1.15], 2.01, False, {'regret': 43, 'year': 1929}]
1 [[0.0], 3.16, False, {'regret': 1, 'year': 1930}]
1 [[-2.67], 7.42, False, {'regret': 0, 'year': 1931}]
3 [[-8.93], -7.41, False, {'regret': 20, 'year': 1932}]
0 [[-10.3], 1.85, False, {'regret': 36, 'year': 1933}]
2 [[-5.19], 7.43, False, {'regret': 51, 'year': 1934}]
3 [[3.48], 14.82, False, {'regret': 0, 'year': 1935}]
2 [[2.55], 1.87, False, {'regret': 41, 'year': 1936}]
1 [[1.03], -0.85, False, {'regret': 31, 'year': 1937}]
2 [[3.73], -2.26, False, {'regret': 0, 'year': 1938}]


We can use this environment in a kind of *monte carlo simulation* to measure a baseline for what the rewards would be over a long period if you always used a random action.

In [14]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    #print(action, context, reward, done, info)

In [15]:
df_mc = pd.DataFrame(reward_list, columns=["reward"])
df_mc.mean()

reward    3.543726
dtype: float64

Depending on the number of iterations, you'll probably get a value approaching 3.75% as a baseline for random actions. That's more than the -5.64% worst case and must less than 15.18% best case for the reward!

In [16]:
plot_line(df_mc, x_col='index', y_col='reward', title='Reward Over Iterations')

([image](../../images/rllib/MarketReward-Random.png))

Yes, it looks quite random... There's no improvement happening at all.

## Training a policy in RLlib

Now let's train a policy using our contextual bandit, specifically using _Linear Thompson Sampling_ in RLlib. Hopefully it will do better than the random results we just computed!

Recall in the `__init__()` method for `MarketBandit` that we set some parameters from the passed in `config` object (with defaults). We don't construct this explicitly ourselves. Rather, RLlib will do this. So, we need to construct the canonical `config` object we want to use. To do this, we use the idioms shown in the next several cells:

In [17]:
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer
import ray

/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paul/anaconda3/e

Initialize Ray as required:

In [18]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [19]:
ray.init(address='auto', ignore_reinit_error=True)

{'node_ip_address': '192.168.1.114',
 'raylet_ip_address': '192.168.1.114',
 'redis_address': '192.168.1.114:6379',
 'object_store_address': '/tmp/ray/session_2020-07-19_18-04-09_292511_3629/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-19_18-04-09_292511_3629/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-19_18-04-09_292511_3629'}

We need a custom config object with our parameters for `MarketBandit`. We do this building on the default `TS_CONFIG` object for _LinTS_:

In [20]:
market_config = with_base_config(TS_CONFIG, {
    "env":           MarketBandit,
    'max-inflation': DEFAULT_MAX_INFLATION,
    'tickers':       DEFAULT_TICKERS,
    'data-file':     DEFAULT_DATA_FILE
})

stop = {
    "training_iteration": 100
}

Also, we'll define a custom trainer, which builds on the `LinTSTrainer`, with "updates". Note that it's the first argument we'll pass to `tune.run()` in the following cell. When all we need is `LinTSTrainer`, as is, and no extra custom config settings, we can just pass the string `contrib/LinTS` to `tune.run()`.  

In [21]:
MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [22]:
analysis = ray.tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=2,              # Change to 0 or 1 to reduce the output.
    ray_auto_init=False,    # Don't allow Tune to initialize Ray.
)

Trial name,status,loc
MarketLinTSTrainer_MarketBandit_7eb5f_00000,RUNNING,
MarketLinTSTrainer_MarketBandit_7eb5f_00001,PENDING,
MarketLinTSTrainer_MarketBandit_7eb5f_00002,PENDING,


(pid=3664) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3664)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3664) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3664)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=3664) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3664)   _np_q

2020-07-19 18:04:41,845	ERROR trial_runner.py:520 -- Trial MarketLinTSTrainer_MarketBandit_7eb5f_00002: Error processing event.
Traceback (most recent call last):
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 468, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/worker.py", line 1474, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(AttributeError): ray::MarketLinTSTrainer.train() (pid=3661, ip=192.168.1.114)
  File "python/ray/_raylet.pyx", line 446, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 400, in ray._raylet.execute_task.function_executor
  File "/home/paul/anaconda3/envs/an

Trial name,status,loc
MarketLinTSTrainer_MarketBandit_7eb5f_00000,ERROR,
MarketLinTSTrainer_MarketBandit_7eb5f_00001,ERROR,
MarketLinTSTrainer_MarketBandit_7eb5f_00002,ERROR,
Trial name,# failures,error file
MarketLinTSTrainer_MarketBandit_7eb5f_00000,1,/home/paul/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-07-19_18-04-39wcarj8p7/error.txt
MarketLinTSTrainer_MarketBandit_7eb5f_00001,1,/home/paul/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_1_2020-07-19_18-04-39vtcbdzs6/error.txt
MarketLinTSTrainer_MarketBandit_7eb5f_00002,1,/home/paul/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_2_2020-07-19_18-04-39mfsmbmce/error.txt


TuneError: ('Trials did not complete', [MarketLinTSTrainer_MarketBandit_7eb5f_00000, MarketLinTSTrainer_MarketBandit_7eb5f_00001, MarketLinTSTrainer_MarketBandit_7eb5f_00002])

## Analyzing the results

Let's analyze the rewards and cumulative regrets of these trials.

In [ ]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

In [ ]:
rewards = df_ts \
    .groupby("info/num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

In [ ]:
plot_line_with_stddev(rewards, x_col='info/num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

([image](../../images/rllib/Market-Bandit-Rewards-vs-Steps.png))

The rewards reach what appears to be nearly optimal by 3000 steps, then shows some slow improvement beyond 8000.

In [ ]:
regrets = df_ts \
    .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

In [ ]:
plot_cumulative_regret(regrets)

([image](../../images/rllib/Market-Bandit-Cumulative-Regret.png))

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [ ]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

You should see a number between 6-8%. That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets continue to grow over time...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [ ]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

In [ ]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

Access the model, to review the distribution of arm weights

In [ ]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

A final note: when you checkpoint it will change how the training performs in this notebook, if you rerun the training! So be sure to start from scratch when doing experiments here, if that's what you intend!